In [ ]:
#!pip install -Uqqq pip
#!pip install -qqq bitsandbytes==0.39.0
#!pip install -qqq torch==2.0.1
#!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
#!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
#!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
#!pip install -qqq datasets==2.12.0
#!pip install -qqq loralib==0.1.1
#!pip install -qqq einops==0.6.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 2.5.0.post0 requires torch>=2.1.0, but you have torch 2.0.1 which is incompatible.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.30.0.dev0 which is incompatible.
torchaudio 2.5.1+cu121 requires torch==2.5.1, but you have torch 2.0.1 which is incompatible.
torchvision 0.20.1+cu121 requires torch==2.5.1, but you have torch 2.0.1 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
# Execute when encoding changes from utf-8 to ANSI for some reason
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [4]:
import json
import os
from pprint import pprint
#!pip uninstall bitsandbytes -y
#!pip install bitsandbytes --no-cache-dir
#!pip install -U triton
#!pip install -U datasets
import torch
print(torch.cuda.is_available())
print(torch.__version__)  # Should return True if CUDA is working
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 112.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: triton
    Found existing installation: triton 2.0.0
    Uninstalling triton-2.0.0:
      Successfully uninstalled triton-2.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.0.1 requires triton==2.0.0; platform_system == "Linux" and platform_machine == "x86_64", but you have triton 3.2.0 which is incompatible.
torchaudio 2.5.1+cu121 requires torch==2.5.1, but you have torch 2.0.1 which is incompatible.
torchvision 0.20.1+cu121 requires torch==2.5.1, but you have torch 2.0.1 which is incompatible.
  Attempting uninstall: datasets
    Found existing installation: datasets 2.12.0
    Uninstalling datasets-2.12.0:
      Successfully uninstalled datasets-2.12.0
True
2.0.1+cu117


ImportError: cannot import name 'ir' from 'triton._C.libtriton' (/usr/local/lib/python3.10/dist-packages/triton/_C/libtriton.so)

In [4]:
notebook_login()

In [5]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

...

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [6]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [7]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [8]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


In [9]:
prompt = """
<human>: Tell me about bullish vs bearish trends
<assistant>:
""".strip()

In [10]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [11]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids = encoding.input_ids,
        
        attention_mask = encoding.attention_mask,
        generation_config = generation_config
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: Tell me about bullish vs bearish trends
<assistant>: Bullish trends are characterized by a rise in prices and an increase in market activity. This is often associated with positive economic news or a shift in market sentiment. Conversely, bearish trends are characterized by a decline in prices and a decrease in market activity. This is often associated with negative economic news or a shift in market sentiment.
User 
CPU times: user 9.61 s, sys: 377 ms, total: 9.98 s
Wall time: 12 s


In [12]:
Reddit_data = load_dataset("csv", data_files="/kaggle/input/stocks/stocks.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [13]:
import pandas as pd
reddit_df = pd.DataFrame(Reddit_data['train'])
print(reddit_df.head())

                                               title  score      id  \
0  Toyota Chief Says ‘Silent Majority’ Has Doubts...     48  zpgxoj   
1  WSJ: Retail/small investors dive into markets ...      5  zpgvhy   
2      Should Elon Musk continue as head of Twitter?      1  zpg589   
3                           Net Income vs Net Margin      7  zpfrun   
4                     Impact of fed funds rate at 5%     54  zpckrj   

                                                 url  comms_num       created  \
0  https://www.reddit.com/r/stocks/comments/zpgxo...         21  1.671421e+09   
1  https://www.reddit.com/r/stocks/comments/zpgvh...          4  1.671421e+09   
2  https://www.reddit.com/r/stocks/comments/zpg58...         50  1.671419e+09   
3  https://www.reddit.com/r/stocks/comments/zpfru...          6  1.671418e+09   
4  https://www.reddit.com/r/stocks/comments/zpckr...         30  1.671409e+09   

                                                body            timestamp  
0  BURIRAM

In [14]:
reddit_df = reddit_df[['title', 'body', 'score', 'comms_num', 'created']]
# Fill missing body text with an empty string
reddit_df['body'].fillna("", inplace=True)

# Drop rows where the title is missing (since it's crucial)
reddit_df.dropna(subset=['title'], inplace=True)

<ipython-input-14-d177b7bcae8d>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reddit_df['body'].fillna("", inplace=True)


In [15]:
from datetime import datetime

reddit_df['created'] = reddit_df['created'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

In [16]:
import re

def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove special characters
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

reddit_df['title'] = reddit_df['title'].apply(clean_text)
reddit_df['body'] = reddit_df['body'].apply(clean_text)
print(reddit_df.head())

                                               title  \
0  toyota chief says silent majority has doubts a...   
1  wsj retailsmall investors dive into markets as...   
2       should elon musk continue as head of twitter   
3                           net income vs net margin   
4                      impact of fed funds rate at 5   

                                                body  score  comms_num  \
0  buriram thailandtoyota motor corp tm 087decrea...     48         21   
1  during the wildest year for global markets sin...      5          4   
2  musk launched a poll on twitter should i step ...      1         50   
3  i ran across a successful hedge fund manager w...      7          6   
4  what is the impact of the terminal rate sittin...     54         30   

               created  
0  2022-12-19 03:41:09  
1  2022-12-19 03:38:12  
2  2022-12-19 03:02:22  
3  2022-12-19 02:44:43  
4  2022-12-19 00:11:54  


In [17]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

nltk.download("vader_lexicon")
sia = SentimentIntensityAnalyzer()

def get_sentiment(text):
    score = sia.polarity_scores(text)['compound']
    return "positive" if score > 0.05 else "negative" if score < -0.05 else "neutral"

reddit_df['sentiment'] = reddit_df['title'].apply(get_sentiment)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


/usr/local/lib/python3.10/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [18]:
reddit_df.to_csv("cleaned_reddit_data.csv", index=False)

In [19]:
cleaned_r_data = load_dataset("csv", data_files="cleaned_reddit_data.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [20]:
cleaned_r_data['train'][0]

{'title': 'toyota chief says silent majority has doubts about pursuing only evs',
 'body': 'buriram thailandtoyota motor corp tm 087decrease red down pointing triangle president akio toyoda said he is among the auto industrys silent majority in questioning whether electric vehicles should be pursued exclusively comments that reflect a growing uneasiness about how quickly car companies can transition auto makers are making big bets on fully electric vehicles investments that have been bolstered by robust demand for the limited numbers of models that are now available still challenges are mountingparticularly in securing parts and raw materials for batteriesand concerns have emerged in some pockets of the car business about the speed to which buyers will make the shift especially as ev prices have soared this year people involved in the auto industry are largely a silent majority mr toyoda said to reporters during a visit to thailand that silent majority is wondering whether evs are real

In [21]:
 def generate_prompt(data_point):
    """
    Generate prompt for fine-tuning based on Reddit stock discussions.
    """
    return f"""
<human>: What is the sentiment of the following stock discussion?
Title: {data_point["title"]}
Body: {data_point["body"]}

<assistant>: {data_point["sentiment"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
    """
    Generate the full prompt and tokenize it.
    """
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

cleaned_r_data = cleaned_r_data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/1743 [00:00<?, ? examples/s]

In [22]:
cleaned_r_data

Dataset({
    features: ['title', 'body', 'score', 'comms_num', 'created', 'sentiment', 'input_ids', 'attention_mask'],
    num_rows: 1743
})

In [ ]:
data = load_dataset("csv", data_files="A.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'],
        num_rows: 5124
    })
})

In [ ]:
import pandas as pd
df = pd.DataFrame(data['train'])

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df = df.sort_values(by='Date')
print(df.isnull().sum())

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


In [ ]:
df = df.dropna()  # Remove missing rows
df = df.drop_duplicates()
from scipy.stats import zscore
df = df[(zscore(df[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]) < 3).all(axis=1)]

print(df.head())

        Date       Open       High        Low      Close  Adj Close   Volume
2 1999-11-22  29.551144  31.473534  28.657009  31.473534  27.068665  6577800
3 1999-11-23  30.400572  31.205294  28.612303  28.612303  24.607880  5975600
4 1999-11-24  28.701717  29.998211  28.612303  29.372318  25.261524  4843200
5 1999-11-26  29.238197  29.685265  29.148785  29.461731  25.338428  1729400
6 1999-11-29  29.327610  30.355865  29.014664  30.132332  25.915169  4074700


In [ ]:
df = pd.read_csv("all-data.csv", encoding="ISO-8859-1")

# Rename columns
df.columns = ["sentiment", "text"]

# Save back as UTF-8 CSV (not xlsx!)
df.to_csv("sentiment_file.csv", index=False, encoding="utf-8")

# Load the cleaned dataset
data2 = load_dataset("csv", data_files="sentiment_file.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
data2

DatasetDict({
    train: Dataset({
        features: ['sentiment', 'text'],
        num_rows: 4845
    })
})

In [ ]:
sentiment_df = pd.DataFrame(data2['train'])
sentiment_df = sentiment_df.dropna()
print(df.head(), sentiment_df.head())

        Date       Open       High        Low      Close  Adj Close   Volume
2 1999-11-22  29.551144  31.473534  28.657009  31.473534  27.068665  6577800
3 1999-11-23  30.400572  31.205294  28.612303  28.612303  24.607880  5975600
4 1999-11-24  28.701717  29.998211  28.612303  29.372318  25.261524  4843200
5 1999-11-26  29.238197  29.685265  29.148785  29.461731  25.338428  1729400
6 1999-11-29  29.327610  30.355865  29.014664  30.132332  25.915169  4074700   sentiment                                               text
0   neutral  Technopolis plans to develop in stages an area...
1  negative  The international electronic industry company ...
2  positive  With the new production plant the company woul...
3  positive  According to the company 's updated strategy f...
4  positive  FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...


In [ ]:
stock_texts = []
for _, row in df.iterrows():
    text = f"[STOCK] Date: {row['Date']}, Open: {row['Open']}, Close: {row['Close']}, Volume: {row['Volume']}\n"
    text += "Question: What is the stock trend?\n"
    if row["Close"] > row["Open"]:
        text += "Answer: The stock closed higher.\n"
    else:
        text += "Answer: The stock closed lower.\n"
    stock_texts.append(text)

sentiment_texts = []
for _, row in sentiment_df.iterrows():
    text = f"[NEWS] Headline: {row['text']}\n"
    text += f"Question: What is the sentiment?\n"
    text += f"Answer: {row['sentiment']}.\n"
    sentiment_texts.append(text)


In [ ]:
import random

final_texts = stock_texts + sentiment_texts  # Combine both datasets
random.shuffle(final_texts)

In [ ]:
with open("training_data.txt", "w", encoding="utf-8") as f:
    for text in final_texts:
        f.write(text + "\n")

In [ ]:
data3 = load_dataset("text", data_files="training_data.txt")

# Check the first few entries
print(data3["train"][0])
print(data3["train"][1])

Generating train split: 0 examples [00:00, ? examples/s]

{'text': '[STOCK] Date: 2002-08-23 00:00:00, Open: 12.060086250305176, Close: 11.459227561950684, Volume: 2544400'}
{'text': 'Question: What is the stock trend?'}


In [ ]:
data3

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 39528
    })
})

In [ ]:
import re
import logging
from datasets import Dataset, DatasetDict

logging.basicConfig(level=logging.INFO)

def combine_entries(text_data):
    """
    Combines individual lines into complete entries.
    Each entry should contain a header ([STOCK] or [NEWS]), a question, and an answer.
    """
    combined_entries = []
    current_entry = []

    for line in text_data:
        line = line.strip()
        if not line:  # Skip empty lines
            continue

        # If we find a new header, store the previous entry (if any) and start a new one
        if line.startswith('[STOCK]') or line.startswith('[NEWS]'):
            if current_entry:
                combined_entries.append('\n'.join(current_entry))
            current_entry = [line]
        else:
            current_entry.append(line)

    # Don't forget to add the last entry
    if current_entry:
        combined_entries.append('\n'.join(current_entry))

    return combined_entries

def preprocess_data_point(data_point):
    """
    Process a single complete entry (header + question + answer).
    """
    text = data_point.get("text", "").strip()

    # Skip empty entries
    if not text:
        return None

    # Initialize processed data structure
    processed_data = {
        "original_text": text,
        "type": None,
        "input_text": None,
        "question": None,
        "answer": None,
        "date": None,
        "open_price": None,
        "close_price": None,
        "volume": None,
        "headline": None,
        "sentiment": None
    }

    lines = text.split('\n')

    # Process Stock Data
    if text.startswith('[STOCK]'):
        stock_match = re.search(
            r"\[STOCK\] Date: (.*?), Open: ([\d.]+), Close: ([\d.]+), Volume: (\d+)",
            lines[0]
        )
        question = next((line for line in lines if line.startswith('Question:')), None)
        answer = next((line for line in lines if line.startswith('Answer:')), None)

        if stock_match and question and answer:
            processed_data.update({
                "type": "stock",
                "date": stock_match.group(1),
                "open_price": float(stock_match.group(2)),
                "close_price": float(stock_match.group(3)),
                "volume": int(stock_match.group(4)),
                "question": question.replace('Question:', '').strip(),
                "answer": answer.replace('Answer:', '').strip(),
                "input_text": lines[0]
            })
            return processed_data

    # Process News Data
    elif text.startswith('[NEWS]'):
        headline_match = re.search(r"\[NEWS\] Headline: (.*?)(?=\nQuestion:|\Z)", text, re.DOTALL)
        question = next((line for line in lines if line.startswith('Question:')), None)
        answer = next((line for line in lines if line.startswith('Answer:')), None)

        if headline_match and question and answer:
            answer_text = answer.replace('Answer:', '').strip()
            processed_data.update({
                "type": "news",
                "headline": headline_match.group(1).strip(),
                "question": question.replace('Question:', '').strip(),
                "answer": answer_text,
                "sentiment": answer_text.replace('.', '').strip(),
                "input_text": lines[0]
            })
            return processed_data

    return None

def process_dataset(raw_data):
    """
    Process the entire dataset, first combining related lines into complete entries.
    """
    try:
        # First, combine the related lines into complete entries
        combined_entries = combine_entries(raw_data["train"]["text"])

        # Create a new dataset with combined entries
        combined_dataset = Dataset.from_dict({"text": combined_entries})

        # Apply preprocessing to combined entries
        processed_dataset = combined_dataset.map(
            preprocess_data_point,
            remove_columns=combined_dataset.column_names,
            desc="Preprocessing data"
        )

        # Remove None entries
        processed_dataset = processed_dataset.filter(lambda x: x is not None)

        # Log processing statistics
        logging.info(f"Original entries: {len(raw_data['train'])}")
        logging.info(f"Combined entries: {len(combined_entries)}")
        logging.info(f"Processed entries: {len(processed_dataset)}")

        return processed_dataset

    except Exception as e:
        logging.error(f"Error during processing: {str(e)}")
        raise

# Main execution code
def main(dataset_dict):
    """
    Main processing pipeline.
    """
    try:
        # Process the training data
        processed_train = process_dataset(dataset_dict)

        # Check if we have any valid processed data
        if len(processed_train) == 0:
            raise ValueError("No valid entries after processing")

        # Create train/test split
        split = processed_train.train_test_split(test_size=0.1, seed=42)

        # Create final dataset dictionary
        final_dataset = DatasetDict({
            "train": split["train"],
            "test": split["test"]
        })

        # Print sample entries
        print("\nSample processed entries:")
        for i in range(min(3, len(final_dataset["train"]))):
            print(f"\nEntry {i+1}:")
            for key, value in final_dataset["train"][i].items():
                print(f"{key}: {value}")

        return final_dataset

    except Exception as e:
        logging.error(f"Error in main processing pipeline: {str(e)}")
        raise
# Set up logging
logging.basicConfig(level=logging.INFO)

try:
    # Process the data
    processed_dataset = main(data3)

    print("\nProcessing completed successfully!")

except Exception as e:
    print(f"An error occurred: {str(e)}")

Preprocessing data:   0%|          | 0/9882 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9882 [00:00<?, ? examples/s]


Sample processed entries:

Entry 1:
original_text: [STOCK] Date: 2004-11-23 00:00:00, Open: 16.380542755126953, Close: 16.330472946166992, Volume: 2979500
Question: What is the stock trend?
Answer: The stock closed lower.
type: stock
input_text: [STOCK] Date: 2004-11-23 00:00:00, Open: 16.380542755126953, Close: 16.330472946166992, Volume: 2979500
question: What is the stock trend?
answer: The stock closed lower.
date: 2004-11-23 00:00:00
open_price: 16.380542755126953
close_price: 16.330472946166992
volume: 2979500
headline: None
sentiment: None

Entry 2:
original_text: [STOCK] Date: 2010-02-25 00:00:00, Open: 22.06008529663086, Close: 22.23891258239746, Volume: 4778500
Question: What is the stock trend?
Answer: The stock closed higher.
type: stock
input_text: [STOCK] Date: 2010-02-25 00:00:00, Open: 22.06008529663086, Close: 22.23891258239746, Volume: 4778500
question: What is the stock trend?
answer: The stock closed higher.
date: 2010-02-25 00:00:00
open_price: 22.06008529663086
c

In [ ]:
def inspect_dataset(dataset):
    """
    Provides a comprehensive inspection of the dataset structure and contents.

    Args:
        dataset: The preprocessed dataset to inspect
    """
    # Dataset Overview
    print("=== Dataset Overview ===")
    print(f"Dataset type: {type(dataset)}")
    print(f"Available splits: {dataset.keys()}\n")

    # Detailed split information
    for split_name, split_data in dataset.items():
        print(f"=== {split_name.upper()} Split Analysis ===")
        print(f"Number of examples: {len(split_data)}")

        # Feature information
        print("\nFeature Information:")
        features = split_data.features
        for feature_name in features:
            print(f"- {feature_name}: {features[feature_name]}")

        # Sample entries
        print("\nFirst 3 entries with formatted structure:")
        for i in range(min(3, len(split_data))):
            print(f"\nEntry {i+1}:")
            entry = split_data[i]
            for key, value in entry.items():
                # Format the output for better readability
                if isinstance(value, (int, float)):
                    print(f"  {key}: {value:,.6f}" if isinstance(value, float) else f"  {key}: {value:,}")
                else:
                    print(f"  {key}: {value}")

        # Type distribution
        print("\nEntry type distribution:")
        type_counts = {}
        for item in split_data:
            entry_type = item['type']
            type_counts[entry_type] = type_counts.get(entry_type, 0) + 1

        for entry_type, count in type_counts.items():
            percentage = (count / len(split_data)) * 100
            print(f"- {entry_type}: {count:,} entries ({percentage:.2f}%)")

        # Numerical field statistics (for stock entries)
        if 'stock' in type_counts:
            print("\nStock data statistics:")
            stock_entries = [item for item in split_data if item['type'] == 'stock']

            numerical_fields = ['open_price', 'close_price', 'volume']
            for field in numerical_fields:
                values = [entry[field] for entry in stock_entries]
                print(f"\n{field}:")
                print(f"  Min: {min(values):,.2f}" if field != 'volume' else f"  Min: {min(values):,}")
                print(f"  Max: {max(values):,.2f}" if field != 'volume' else f"  Max: {max(values):,}")
                print(f"  Average: {sum(values)/len(values):,.2f}" if field != 'volume' else f"  Average: {sum(values)/len(values):,.0f}")

# Use the inspection utility
try:
    print("Inspecting preprocessed dataset...\n")
    inspect_dataset(processed_dataset)

except Exception as e:
    print(f"An error occurred during inspection: {str(e)}")

inspect_dataset(processed_dataset)


Inspecting preprocessed dataset...

=== Dataset Overview ===
Dataset type: <class 'datasets.dataset_dict.DatasetDict'>
Available splits: dict_keys(['train', 'test'])

=== TRAIN Split Analysis ===
Number of examples: 8893

Feature Information:
- original_text: Value(dtype='string', id=None)
- type: Value(dtype='string', id=None)
- input_text: Value(dtype='string', id=None)
- question: Value(dtype='string', id=None)
- answer: Value(dtype='string', id=None)
- date: Value(dtype='string', id=None)
- open_price: Value(dtype='float64', id=None)
- close_price: Value(dtype='float64', id=None)
- volume: Value(dtype='int64', id=None)
- headline: Value(dtype='string', id=None)
- sentiment: Value(dtype='string', id=None)

First 3 entries with formatted structure:

Entry 1:
  original_text: [STOCK] Date: 2004-11-23 00:00:00, Open: 16.380542755126953, Close: 16.330472946166992, Volume: 2979500
Question: What is the stock trend?
Answer: The stock closed lower.
  type: stock
  input_text: [STOCK] Date:

In [ ]:
print(processed_dataset['train'][0])
print(processed_dataset['train'][1000])

{'original_text': '[STOCK] Date: 2004-11-23 00:00:00, Open: 16.380542755126953, Close: 16.330472946166992, Volume: 2979500\nQuestion: What is the stock trend?\nAnswer: The stock closed lower.', 'type': 'stock', 'input_text': '[STOCK] Date: 2004-11-23 00:00:00, Open: 16.380542755126953, Close: 16.330472946166992, Volume: 2979500', 'question': 'What is the stock trend?', 'answer': 'The stock closed lower.', 'date': '2004-11-23 00:00:00', 'open_price': 16.380542755126953, 'close_price': 16.330472946166992, 'volume': 2979500, 'headline': None, 'sentiment': None}
{'original_text': '[NEWS] Headline: BasWare is headquartered in Espoo , Finland .\nQuestion: What is the sentiment?\nAnswer: neutral.', 'type': 'news', 'input_text': '[NEWS] Headline: BasWare is headquartered in Espoo , Finland .', 'question': 'What is the sentiment?', 'answer': 'neutral.', 'date': None, 'open_price': None, 'close_price': None, 'volume': None, 'headline': 'BasWare is headquartered in Espoo , Finland .', 'sentiment'

In [ ]:
def generate_prompt(data_point):
    # Handle news data
    if data_point["type"] == "news_data":
        return f"""
<human>: Here is a financial news headline:
Headline: {data_point['headline']}

What sentiment does this news convey, and how might it impact stock investments?
<assistant>: The sentiment of this news is {data_point['sentiment']}.
""".strip()

    # Handle stock data
    elif data_point["type"] == "stock_data":
        return f"""
<human>: Based on the following stock market data:
Date: {data_point['date']}
Open: {data_point['open_price']}
Close: {data_point['close_price']}
Volume: {data_point['volume']}

What investment advice would you provide?
<assistant>:
""".strip()

    # Handle questions
    elif data_point["type"] == "question":
        return f"""
<human>: {data_point['question']}
<assistant>:
""".strip()

    # Handle answers
    elif data_point["type"] == "unknown" and data_point["answer"] is not None:
        return f"""
<human>: {data_point['text']}
<assistant>: {data_point['answer']}
""".strip()

    # Default case for unknown or unsupported types
    else:
        return None  # Skip this data point

from transformers import AutoTokenizer

# Load a tokenizer (change to your desired model)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    if full_prompt is not None:  # Only tokenize if a valid prompt is generated
        return tokenizer(full_prompt, padding=True, truncation=True)
    else:
        return None  # Skip this data point

# Filter out None values after tokenization
def filter_none(batch):
    return {k: [v for v in batch[k] if v is not None] for k in batch}

# Shuffle and tokenize the structured data
structured_data = structured_data.shuffle().map(generate_and_tokenize_prompt, batched=True).filter(filter_none, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/39528 [00:00<?, ? examples/s]

Filter:   0%|          | 0/39528 [00:00<?, ? examples/s]

In [ ]:
structured_data

Dataset({
    features: ['text', 'type', 'is_stock_data', 'date', 'open_price', 'close_price', 'volume', 'is_question', 'question', 'is_news_data', 'headline', 'sentiment', 'answer'],
    num_rows: 39528
})

In [ ]:
for batch in train_dataloader:
    print(batch)
    break  # Only print the first batch


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'train_dataloader' is not defined

In [24]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=1e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=10,
    output_dir="stock_advice_experiments",
    optim="adamw_torch",
    lr_scheduler_type="linear",
    warmup_ratio=0.1,
)

# Train the Model with PEFT
trainer = transformers.Trainer(
    model=model,
    train_dataset=cleaned_r_data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

# Disable cache for training
model.config.use_cache = False
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,3.969700
20,3.873300
30,3.806800
40,3.408800
50,3.028600
60,2.758200
70,2.695100
80,2.620400
90,2.622100
100,2.699100


TrainOutput(global_step=327, training_loss=2.7022304709898224, metrics={'train_runtime': 9144.3217, 'train_samples_per_second': 0.572, 'train_steps_per_second': 0.036, 'total_flos': 2.6379687678829056e+16, 'train_loss': 2.7022304709898224, 'epoch': 3.0})

In [23]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [25]:
model.save_pretrained("trained-model")

In [27]:
import os
print(os.path.abspath("trained-model"))

/kaggle/working/trained-model


In [30]:
PEFT_MODEL = "Muhammad2328E/Invest-falcon-7b"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Muhammad2328E/Invest-falcon-7b/commit/a60144ee647f737b6b8b2ad8ca9157b3dc417a83', commit_message='Upload model', commit_description='', oid='a60144ee647f737b6b8b2ad8ca9157b3dc417a83', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Muhammad2328E/Invest-falcon-7b', endpoint='https://huggingface.co', repo_type='model', repo_id='Muhammad2328E/Invest-falcon-7b'), pr_revision=None, pr_num=None)

In [12]:
# Import necessary modules if you haven't already
import torch
from transformers import BitsAndBytesConfig

# Create the quantization configuration with CPU offloading enabled
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,  # This helps reduce memory usage further
    llm_int8_enable_fp32_cpu_offload=True  # Enable CPU offloading
)

# For the device map, we should be more systematic about layer distribution
# Let's create a function to help generate a more balanced device map
def create_device_map(num_layers):
    # Calculate how many layers to keep on GPU vs CPU based on your memory constraints
    # This is a simple example - you might need to adjust the ratio
    gpu_layers = int(num_layers * 0.7)  # Keep 70% on GPU
    
    device_map = {
        "model.embed_tokens": "cuda:0",
        "model.norm": "cuda:0",
        "lm_head": "cuda:0"
    }
    
    # Distribute the transformer layers
    for i in range(num_layers):
        if i < gpu_layers:
            device_map[f"model.layers.{i}"] = "cuda:0"
        else:
            device_map[f"model.layers.{i}"] = "cpu"
    
    return device_map

# Now load the model with our corrected configuration
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",  # Let's start with automatic distribution
    trust_remote_code=True,
    torch_dtype=torch.float16  # Ensure we're using half precision throughout
)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [23]:
%%time
device = "cuda:0"

prompt = """
<human>: What’s the consensus on growth vs. value stocks in 2025?
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: What’s the consensus on growth vs. value stocks in 2025?
<assistant>: According to recent market trends, growth stocks are expected to outperform value stocks in 2025.
User 
CPU times: user 1.79 s, sys: 0 ns, total: 1.79 s
Wall time: 1.78 s


In [37]:
!zip -r trained-model.zip trained-model

  adding: trained-model/ (stored 0%)
  adding: trained-model/adapter_config.json (deflated 42%)
  adding: trained-model/adapter_model.bin (deflated 8%)


In [38]:
!mv trained-model.zip /kaggle/outputs/

mv: cannot create regular file '/kaggle/outputs/': Not a directory


In [44]:
!pip install huggingface_hub

In [46]:
from huggingface_hub import snapshot_download
import time
import os
from pathlib import Path

def download_model_with_retry(
    repo_id,
    local_dir,
    max_retries=5,
    retry_delay=10,
    timeout=60
):
    """
    Downloads a model from Hugging Face Hub with retry logic for better reliability.
    
    Args:
        repo_id (str): The Hugging Face model repository ID
        local_dir (str): Local directory to save the model
        max_retries (int): Maximum number of retry attempts
        retry_delay (int): Delay in seconds between retries
        timeout (int): Timeout in seconds for each download attempt
    
    Returns:
        str: Path to the downloaded model
    """
    # Create the directory if it doesn't exist
    Path(local_dir).mkdir(parents=True, exist_ok=True)
    
    for attempt in range(max_retries):
        try:
            print(f"Download attempt {attempt + 1} of {max_retries}")
            
            # Attempt to download with increased timeout
            local_path = snapshot_download(
                repo_id=repo_id,
                local_dir=local_dir,
                local_dir_use_symlinks=False,
                token=True,  # Will use token if you're logged in
                ignore_patterns=["*.msgpack", "*.h5", "*.safetensors"],  # Optional: ignore large files
                max_workers=1,  # Reduce parallel downloads to improve stability
                retry_on_error=True,  # Enable built-in retries
                request_timeout=timeout  # Increase timeout period
            )
            
            print(f"Successfully downloaded model to: {local_path}")
            return local_path
            
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"Download failed with error: {str(e)}")
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                print("Maximum retry attempts reached. Download failed.")
                raise

# Use the function with your model
try:
    model_path = download_model_with_retry(
        repo_id="Muhammad2328E/Invest-falcon-7b",
        local_dir="Downloads",
        timeout=120  # Increased timeout to 2 minutes
    )
    print(f"Model successfully downloaded to: {model_path}")
except Exception as e:
    print(f"Failed to download model: {str(e)}")

Download attempt 1 of 5
Download failed with error: snapshot_download() got an unexpected keyword argument 'retry_on_error'
Retrying in 10 seconds...
Download attempt 2 of 5
Download failed with error: snapshot_download() got an unexpected keyword argument 'retry_on_error'
Retrying in 10 seconds...
Download attempt 3 of 5
Download failed with error: snapshot_download() got an unexpected keyword argument 'retry_on_error'
Retrying in 10 seconds...
Download attempt 4 of 5
Download failed with error: snapshot_download() got an unexpected keyword argument 'retry_on_error'
Retrying in 10 seconds...
Download attempt 5 of 5
Maximum retry attempts reached. Download failed.
Failed to download model: snapshot_download() got an unexpected keyword argument 'retry_on_error'
